In [1]:
folder = "datasets/p19/rawdata/training_setA"


In [2]:
import pandas as pd
fpath = "../datasets/p19/rawdata/training_setA/p000001.psv"

data = pd.read_csv(fpath, sep="|", header=0)
print(data.columns)


Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel'],
      dtype='object')


In [3]:
import pandas as pd
fpath = "../datasets/p19/rawdata/training_setB/p100001.psv"

data = pd.read_csv(fpath, sep="|", header=0)
print(data.columns)


Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel'],
      dtype='object')


In [4]:
temp_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2', 'BaseExcess', 'HCO3',
    'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium',
    'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate',
    'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct',
    'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets']

static_columns = ['Age', 'Gender', 'Unit1', 'Unit2']

sepsis_column = 'SepsisLabel'

time_column = 'ICULOS'

print(len(temp_columns), len(static_columns))

34 4


In [5]:
import pandas as pd
fpath = "../datasets/p19/rawdata/training_setA/p000001.psv"

data = pd.read_csv(fpath, sep="|", header=0)
tempX = data[temp_columns]
print(tempX.shape)
staticX = data[static_columns]
print(staticX.shape)
sepsis = data[sepsis_column]
print(sepsis.shape)


(54, 34)
(54, 4)
(54,)


In [6]:
import os
from tqdm import tqdm
import numpy as np

PTdict_list = []
outcome_list = []

folderA = "../datasets/p19/rawdata/training_setA/"


for fname in tqdm(os.listdir(folderA)):
    if not fname.endswith('.psv'):
        continue

    fpath = f"{folderA}/{fname}"

    data = pd.read_csv(fpath, sep="|", header=0)
    tempX = data[temp_columns].fillna(0).values
    staticX = data[static_columns].fillna(0).values[0]
    pid = int(fname[1:-4])
    Tarr = data[time_column].fillna(0).values.reshape((-1, 1))
    sepsis = data[sepsis_column].fillna(0).values

    # y = 1 if np.any(sepsis) else 0
    sepidx = -1
    for idx in range(len(sepsis)-1, 0, -1):
        if sepsis[idx] == 1:
            sepidx = idx
    
    if sepidx == -1 and tempX.shape[0] > 60:
        continue

    if 0 <= sepidx < 12:
        continue

    y = 0
    if sepidx >= 12:
        onset, offset = max(0, sepidx-6-60), sepidx-6
        tempX = tempX[onset:offset, :]
        Tarr = Tarr[onset:offset, :]
        y = 1

    length = tempX.shape[0]
    if length < 60:
        tempX = np.concatenate((tempX, np.zeros((60-length, tempX.shape[1]))))
        Tarr = np.concatenate((Tarr, np.zeros((60-length, Tarr.shape[1]))))

    Pdict = {
        'id': pid,
        'extended_static': staticX,
        'arr': tempX,
        'time': Tarr,
        # 'sepsis': sepsis,
        'length': length
    }
    

    PTdict_list.append(Pdict)
    outcome_list.append(y)


  0%|          | 0/20337 [00:00<?, ?it/s]

100%|██████████| 20337/20337 [00:51<00:00, 392.53it/s]


In [7]:

folderB = "../datasets/p19/rawdata/training_setB/"


for fname in tqdm(os.listdir(folderB)):
    if not fname.endswith('.psv'):
        continue

    fpath = f"{folderB}/{fname}"

    data = pd.read_csv(fpath, sep="|", header=0)
    tempX = data[temp_columns].fillna(0).values
    staticX = data[static_columns].fillna(0).values[0]
    pid = int(fname[1:-4]) + 100000
    Tarr = data[time_column].fillna(0).values.reshape((-1, 1))
    sepsis = data[sepsis_column].fillna(0).values

    # y = 1 if np.any(sepsis) else 0
    sepidx = -1
    for idx in range(len(sepsis)-1, 0, -1):
        if sepsis[idx] == 1:
            sepidx = idx
    
    if sepidx == -1 and tempX.shape[0] > 60:
        continue

    if 0 <= sepidx < 12:
        continue

    y = 0
    if sepidx >= 12:
        onset, offset = max(0, sepidx-6-60), sepidx-6
        tempX = tempX[onset:offset, :]
        Tarr = Tarr[onset:offset, :]
        y = 1

    length = tempX.shape[0]
    if length < 60:
        tempX = np.concatenate((tempX, np.zeros((60-length, tempX.shape[1]))))
        Tarr = np.concatenate((Tarr, np.zeros((60-length, Tarr.shape[1]))))

    Pdict = {
        'id': pid,
        'extended_static': staticX,
        'arr': tempX,
        'time': Tarr,
        # 'sepsis': sepsis,
        'length': length
    }
    

    PTdict_list.append(Pdict)
    outcome_list.append(y)



100%|██████████| 20001/20001 [00:49<00:00, 407.73it/s]


In [8]:
print(len(PTdict_list), len(outcome_list), sum(outcome_list) / len(outcome_list))
np.save('../datasets/p19/processed_data/PTdict_list.npy', PTdict_list)
np.save('../datasets/p19/processed_data/arr_outcomes.npy', outcome_list)

39119 39119 0.04956670671540683


In [9]:
sum(outcome_list)

1939

In [10]:
import pickle as pkl
from torch_geometric.seed import seed_everything


"""Use 8:1:1 split"""
p_train = 0.80
p_val = 0.10
p_test = 0.10

os.makedirs("../datasets/p19/splits", exist_ok=True)

n = len(PTdict_list)  # original 12000 patients, remove 12 outliers
n_train = round(n*p_train)
n_val = round(n*p_val)
n_test = n - (n_train+n_val)
Nsplits = 5

for seed in range(10):
    seed_everything(seed)
    for split in range(Nsplits):
        p = np.random.permutation(n)
        idx_train = p[:n_train]
        idx_val = p[n_train:n_train+n_val]
        idx_test = p[n_train+n_val:]
        with open(f'../datasets/p19/splits/seed{seed}_split{split}.pkl', 'wb') as wbfile:
            print(f'seed{seed}_split{split}', len(idx_train), len(idx_val), len(idx_test))
            pkl.dump((idx_train, idx_val, idx_test), wbfile)

print('split IDs saved')


seed0_split0 31295 3912 3912
seed0_split1 31295 3912 3912
seed0_split2 31295 3912 3912
seed0_split3 31295 3912 3912
seed0_split4 31295 3912 3912
seed1_split0 31295 3912 3912
seed1_split1 31295 3912 3912
seed1_split2 31295 3912 3912
seed1_split3 31295 3912 3912
seed1_split4 31295 3912 3912
seed2_split0 31295 3912 3912
seed2_split1 31295 3912 3912
seed2_split2 31295 3912 3912
seed2_split3 31295 3912 3912
seed2_split4 31295 3912 3912
seed3_split0 31295 3912 3912
seed3_split1 31295 3912 3912
seed3_split2 31295 3912 3912
seed3_split3 31295 3912 3912
seed3_split4 31295 3912 3912
seed4_split0 31295 3912 3912
seed4_split1 31295 3912 3912
seed4_split2 31295 3912 3912
seed4_split3 31295 3912 3912
seed4_split4 31295 3912 3912
seed5_split0 31295 3912 3912
seed5_split1 31295 3912 3912
seed5_split2 31295 3912 3912
seed5_split3 31295 3912 3912
seed5_split4 31295 3912 3912
seed6_split0 31295 3912 3912
seed6_split1 31295 3912 3912
seed6_split2 31295 3912 3912
seed6_split3 31295 3912 3912
seed6_split4 3

In [11]:
PTdict_list = np.load('../datasets/p19/processed_data/PTdict_list.npy', allow_pickle=True)

length_list = [
    PTdict['arr'].shape[0]
    for PTdict in PTdict_list
]
print(max(length_list), min(length_list))


60 60
